# Compare WG Projected Precipitation Depths

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats
import datetime as dt
import sqlalchemy
import seaborn as sns
from copy import deepcopy
import pickle

In [3]:
# input information for weather generators
WG_IN_DIR = r'C:\Temp\WG_Test_Out\Test3\Processed'
OUT_DIR = r'C:\Temp\WG_Test_Out\Test3\Processed\Comparison'
OUT_ROOT = "DC_WGMN3"
H1_DATA_ROOT = "H1_Data_Depth_G"
H1_PROJ1_ROOT = "H1P1_Depth_G"
H1_PROJ2_ROOT = "H1P2_Depth_G"
H1_PROJ3_ROOT = "H1P3_Depth_G"
PLOT_DIR = "Plots_ProjComp"

In [4]:
LOCA_KEYS = [ 62, 63, 64, 76, 77, 78, 79, 90, 91, 92, 93, 94,
              104, 105, 106, 107, 108, 120, 121, 122, 123, 137 ]
NUM_LOCA_GRID = len( LOCA_KEYS )

Start working with the data period.
Load all grid cells and concatenate from the WG

In [5]:
DFList = list()

In [6]:
for gG in LOCA_KEYS:
    cFile = "%s%d.pickle" % ( H1_DATA_ROOT, gG )
    InFP = os.path.normpath( os.path.join( WG_IN_DIR, cFile ) )
    cDF = pd.read_pickle( InFP )
    cDF['Grid_Id'] = gG
    cDF.drop( columns=['Year', 'Day'], inplace=True )
    DFList.append( cDF )
# end of for

Now concatenate

In [7]:
NumDF = len( DFList )
NumDF

22

In [8]:
H1Data = pd.concat( DFList, ignore_index=True )

In [9]:
display( HTML( H1Data.head().to_html() ) )

,Month,PDepth_mm,Grid_Id
0,1,0.903307,62
1,1,11.131537,62
2,1,5.670500,62
3,2,4.555808,62
4,2,3.614461,62


Can now drop some columns

Now filter all into monthly values

In [10]:
H1DJanDF = H1Data[H1Data['Month'] == 1].copy()
H1DFebDF = H1Data[H1Data['Month'] == 2].copy()
H1DMarDF = H1Data[H1Data['Month'] == 3].copy()
H1DAprDF = H1Data[H1Data['Month'] == 4].copy()
H1DMayDF = H1Data[H1Data['Month'] == 5].copy()
H1DJunDF = H1Data[H1Data['Month'] == 6].copy()
H1DJulDF = H1Data[H1Data['Month'] == 7].copy()
H1DAugDF = H1Data[H1Data['Month'] == 8].copy()
H1DSepDF = H1Data[H1Data['Month'] == 9].copy()
H1DOctDF = H1Data[H1Data['Month'] == 10].copy()
H1DNovDF = H1Data[H1Data['Month'] == 11].copy()
H1DDecDF = H1Data[H1Data['Month'] == 12].copy()

In [11]:
del H1Data

In [12]:
MonthLister = [ "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", 
                 "Sep", "Oct", "Nov", "Dec" ]
AveLister = list()
VarLister = list()
SkewLister = list()
KurtLister = list()
CntLister = list()
MedianLister = list()
MaxLister = list()
MinLister = list()

In [13]:
OurQList = [ round( 0.01 * x, 2 ) for x in range(100) ]
OurQList.append( 0.995 )
OurQList.append( 0.999 )
OurQList.append( 0.9999 )
OurQList.append( 1.0 )
OurQList[95:]

[0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999, 0.9999, 1.0]

In [14]:
OurQs = np.array( OurQList, dtype=np.float32 )
NumQs = len( OurQs )
NumQs

104

## Full Dataset Quantiles

In [15]:
DataDict = { MonthLister[0] : np.array( H1DJanDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[1] : np.array( H1DFebDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[2] : np.array( H1DMarDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[3] : np.array( H1DAprDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[4] : np.array( H1DMayDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[5] : np.array( H1DJunDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[6] : np.array( H1DJulDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[7] : np.array( H1DAugDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[8] : np.array( H1DSepDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[9] : np.array( H1DOctDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[10] : np.array( H1DNovDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
             MonthLister[11] : np.array( H1DDecDF['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ), }
H1DataMonthsPDF = pd.DataFrame( index=OurQs, data=DataDict )

In [16]:
AveLister = [ [ H1DJanDF['PDepth_mm'].mean() ],
              [ H1DFebDF['PDepth_mm'].mean() ],
              [ H1DMarDF['PDepth_mm'].mean() ],
              [ H1DAprDF['PDepth_mm'].mean() ],
              [ H1DMayDF['PDepth_mm'].mean() ],
              [ H1DJunDF['PDepth_mm'].mean() ],
              [ H1DJulDF['PDepth_mm'].mean() ],
              [ H1DAugDF['PDepth_mm'].mean() ],
              [ H1DSepDF['PDepth_mm'].mean() ],
              [ H1DOctDF['PDepth_mm'].mean() ],
              [ H1DNovDF['PDepth_mm'].mean() ],
              [ H1DDecDF['PDepth_mm'].mean() ],
            ]
MedianLister = [ [ H1DJanDF['PDepth_mm'].median() ],
              [ H1DFebDF['PDepth_mm'].median() ],
              [ H1DMarDF['PDepth_mm'].median() ],
              [ H1DAprDF['PDepth_mm'].median() ],
              [ H1DMayDF['PDepth_mm'].median() ],
              [ H1DJunDF['PDepth_mm'].median() ],
              [ H1DJulDF['PDepth_mm'].median() ],
              [ H1DAugDF['PDepth_mm'].median() ],
              [ H1DSepDF['PDepth_mm'].median() ],
              [ H1DOctDF['PDepth_mm'].median() ],
              [ H1DNovDF['PDepth_mm'].median() ],
              [ H1DDecDF['PDepth_mm'].median() ],
            ]
MaxLister = [ [ H1DJanDF['PDepth_mm'].max() ],
              [ H1DFebDF['PDepth_mm'].max() ],
              [ H1DMarDF['PDepth_mm'].max() ],
              [ H1DAprDF['PDepth_mm'].max() ],
              [ H1DMayDF['PDepth_mm'].max() ],
              [ H1DJunDF['PDepth_mm'].max() ],
              [ H1DJulDF['PDepth_mm'].max() ],
              [ H1DAugDF['PDepth_mm'].max() ],
              [ H1DSepDF['PDepth_mm'].max() ],
              [ H1DOctDF['PDepth_mm'].max() ],
              [ H1DNovDF['PDepth_mm'].max() ],
              [ H1DDecDF['PDepth_mm'].max() ],
            ]
MinLister = [ [ H1DJanDF['PDepth_mm'].min() ],
              [ H1DFebDF['PDepth_mm'].min() ],
              [ H1DMarDF['PDepth_mm'].min() ],
              [ H1DAprDF['PDepth_mm'].min() ],
              [ H1DMayDF['PDepth_mm'].min() ],
              [ H1DJunDF['PDepth_mm'].min() ],
              [ H1DJulDF['PDepth_mm'].min() ],
              [ H1DAugDF['PDepth_mm'].min() ],
              [ H1DSepDF['PDepth_mm'].min() ],
              [ H1DOctDF['PDepth_mm'].min() ],
              [ H1DNovDF['PDepth_mm'].min() ],
              [ H1DDecDF['PDepth_mm'].min() ],
            ]
VarLister = [ [ H1DJanDF['PDepth_mm'].var() ],
              [ H1DFebDF['PDepth_mm'].var() ],
              [ H1DMarDF['PDepth_mm'].var() ],
              [ H1DAprDF['PDepth_mm'].var() ],
              [ H1DMayDF['PDepth_mm'].var() ],
              [ H1DJunDF['PDepth_mm'].var() ],
              [ H1DJulDF['PDepth_mm'].var()],
              [ H1DAugDF['PDepth_mm'].var() ],
              [ H1DSepDF['PDepth_mm'].var() ],
              [ H1DOctDF['PDepth_mm'].var() ],
              [ H1DNovDF['PDepth_mm'].var() ],
              [ H1DDecDF['PDepth_mm'].var() ],
            ]
SkewLister = [ [ H1DJanDF['PDepth_mm'].skew() ],
              [ H1DFebDF['PDepth_mm'].skew() ],
              [ H1DMarDF['PDepth_mm'].skew() ],
              [ H1DAprDF['PDepth_mm'].skew() ],
              [ H1DMayDF['PDepth_mm'].skew() ],
              [ H1DJunDF['PDepth_mm'].skew() ],
              [ H1DJulDF['PDepth_mm'].skew() ],
              [ H1DAugDF['PDepth_mm'].skew() ],
              [ H1DSepDF['PDepth_mm'].skew() ],
              [ H1DOctDF['PDepth_mm'].skew() ],
              [ H1DNovDF['PDepth_mm'].skew() ],
              [ H1DDecDF['PDepth_mm'].skew() ],
            ]
KurtLister = [ [ H1DJanDF['PDepth_mm'].kurtosis() ],
              [ H1DFebDF['PDepth_mm'].kurtosis() ],
              [ H1DMarDF['PDepth_mm'].kurtosis() ],
              [ H1DAprDF['PDepth_mm'].kurtosis() ],
              [ H1DMayDF['PDepth_mm'].kurtosis() ],
              [ H1DJunDF['PDepth_mm'].kurtosis() ],
              [ H1DJulDF['PDepth_mm'].kurtosis() ],
              [ H1DAugDF['PDepth_mm'].kurtosis() ],
              [ H1DSepDF['PDepth_mm'].kurtosis() ],
              [ H1DOctDF['PDepth_mm'].kurtosis() ],
              [ H1DNovDF['PDepth_mm'].kurtosis() ],
              [ H1DDecDF['PDepth_mm'].kurtosis() ],
            ]
CntLister = [ [ H1DJanDF['PDepth_mm'].count() ],
              [ H1DFebDF['PDepth_mm'].count() ],
              [ H1DMarDF['PDepth_mm'].count() ],
              [ H1DAprDF['PDepth_mm'].count() ],
              [ H1DMayDF['PDepth_mm'].count() ],
              [ H1DJunDF['PDepth_mm'].count() ],
              [ H1DJulDF['PDepth_mm'].count() ],
              [ H1DAugDF['PDepth_mm'].count() ],
              [ H1DSepDF['PDepth_mm'].count() ],
              [ H1DOctDF['PDepth_mm'].count() ],
              [ H1DNovDF['PDepth_mm'].count() ],
              [ H1DDecDF['PDepth_mm'].count() ],
            ]
# process stats to dataframe
DataDict = dict()
DataDict["H1D_Ave"] = np.array( [x[0] for x in AveLister], np.float32 )
DataDict["H1D_Med"] = np.array( [x[0] for x in MedianLister], np.float32 )
DataDict["H1D_Max"] = np.array( [x[0] for x in MaxLister], np.float32 )
DataDict["H1D_Min"] = np.array( [x[0] for x in MinLister], np.float32 )   
DataDict["H1D_Var"] = np.array( [x[0] for x in VarLister], np.float32 )
DataDict["H1D_Skew"] = np.array( [x[0] for x in SkewLister], np.float32 )
DataDict["H1D_Kurt"] = np.array( [x[0] for x in KurtLister], np.float32 )
DataDict["H1D_Cnt"] = np.array( [x[0] for x in CntLister], np.float32 )
SumCompDF = pd.DataFrame( index=MonthLister, data=DataDict )

Output

In [17]:
OutFileFP = os.path.normpath( os.path.join( OUT_DIR, "%s_WG_PDepth_Projections.xlsx" % OUT_ROOT ) )
with pd.ExcelWriter( OutFileFP, engine="openpyxl", mode='w' ) as writer:
    SumCompDF.to_excel( writer, sheet_name="H1D_Summary" )
    H1DataMonthsPDF.to_excel( writer, sheet_name="H1Data_ptiles" )
# end of with block

## Individual Grid Cells

Now need to create some data structures so that can do our processing and analysis

In [18]:
H1DDict = dict()

In [19]:
for gG in LOCA_KEYS:
    InnerDD = dict()
    # now manually assign
    InnerDD[1] = H1DJanDF[H1DJanDF['Grid_Id'] == gG].copy()
    InnerDD[2] = H1DFebDF[H1DFebDF['Grid_Id'] == gG].copy()
    InnerDD[3] = H1DMarDF[H1DMarDF['Grid_Id'] == gG].copy()
    InnerDD[4] = H1DAprDF[H1DAprDF['Grid_Id'] == gG].copy()
    InnerDD[5] = H1DMayDF[H1DMayDF['Grid_Id'] == gG].copy()
    InnerDD[6] = H1DJunDF[H1DJunDF['Grid_Id'] == gG].copy()
    InnerDD[7] = H1DJulDF[H1DJulDF['Grid_Id'] == gG].copy()
    InnerDD[8] = H1DAugDF[H1DAugDF['Grid_Id'] == gG].copy()
    InnerDD[9] = H1DSepDF[H1DSepDF['Grid_Id'] == gG].copy()
    InnerDD[10] = H1DOctDF[H1DOctDF['Grid_Id'] == gG].copy()
    InnerDD[11] = H1DNovDF[H1DNovDF['Grid_Id'] == gG].copy()
    InnerDD[12] = H1DDecDF[H1DDecDF['Grid_Id'] == gG].copy()
    # now assign to our grid dictionary
    H1DDict[gG] = InnerDD
# end of outer for

In [20]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Jan.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DJanDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [21]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Feb.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DFebDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [22]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Mar.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DMarDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [23]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Apr.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DAprDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [24]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_May.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DMayDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [25]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Jun.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DJunDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [26]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Jul.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DJulDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [27]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Aug.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DAugDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [28]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Sep.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DSepDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [29]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Oct.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DOctDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [30]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Nov.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DNovDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [31]:
# now do the month arrays
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_Dec.pickle" % OUT_ROOT ) )
with open( OutFiler, 'wb' ) as OuP:
    pickle.dump( H1DDecDF, OuP, protocol=pickle.HIGHEST_PROTOCOL )

In [32]:
del H1DJanDF
del H1DFebDF
del H1DMarDF
del H1DAprDF
del H1DMayDF
del H1DJunDF
del H1DJulDF
del H1DAugDF
del H1DSepDF
del H1DOctDF
del H1DNovDF
del H1DDecDF

Collate the monthly statistics and output to a spreadsheet

In [33]:
for gG in LOCA_KEYS:
    # initialize our lists
    AveLister = list()
    VarLister = list()
    SkewLister = list()
    KurtLister = list()
    CntLister = list()
    MeanLister = list()
    MaxLister = list()
    MinLister = list()
    # H1 data
    gDDict = H1DDict[gG]
    DataDict = { MonthLister[0] : np.array( gDDict[1]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[1] : np.array( gDDict[2]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[2] : np.array( gDDict[3]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[3] : np.array( gDDict[4]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[4] : np.array( gDDict[5]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[5] : np.array( gDDict[6]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[6] : np.array( gDDict[7]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[7] : np.array( gDDict[8]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[8] : np.array( gDDict[9]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[9] : np.array( gDDict[10]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[10] : np.array( gDDict[11]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ),
                 MonthLister[11] : np.array( gDDict[12]['PDepth_mm'].quantile(q=OurQs), dtype=np.float64 ), }
    DaQsDF = pd.DataFrame( index=OurQs, data=DataDict )
    # do our stats
    # mean
    AveLister = [ [ gDDict[1]['PDepth_mm'].mean() ],
                  [ gDDict[2]['PDepth_mm'].mean() ],
                  [ gDDict[3]['PDepth_mm'].mean() ],
                  [ gDDict[4]['PDepth_mm'].mean() ],
                  [ gDDict[5]['PDepth_mm'].mean() ],
                  [ gDDict[6]['PDepth_mm'].mean() ],
                  [ gDDict[7]['PDepth_mm'].mean() ],
                  [ gDDict[8]['PDepth_mm'].mean() ],
                  [ gDDict[9]['PDepth_mm'].mean() ],
                  [ gDDict[10]['PDepth_mm'].mean() ],
                  [ gDDict[11]['PDepth_mm'].mean() ],
                  [ gDDict[12]['PDepth_mm'].mean() ],
                ]
    # median
    MedianLister = [ [ gDDict[1]['PDepth_mm'].median() ],
                  [ gDDict[2]['PDepth_mm'].median() ],
                  [ gDDict[3]['PDepth_mm'].median() ],
                  [ gDDict[4]['PDepth_mm'].median() ],
                  [ gDDict[5]['PDepth_mm'].median() ],
                  [ gDDict[6]['PDepth_mm'].median() ],
                  [ gDDict[7]['PDepth_mm'].median() ],
                  [ gDDict[8]['PDepth_mm'].median() ],
                  [ gDDict[9]['PDepth_mm'].median() ],
                  [ gDDict[10]['PDepth_mm'].median() ],
                  [ gDDict[11]['PDepth_mm'].median() ],
                  [ gDDict[12]['PDepth_mm'].median() ],
                ]
    # max
    MaxLister = [ [ gDDict[1]['PDepth_mm'].max() ],
                  [ gDDict[2]['PDepth_mm'].max() ],
                  [ gDDict[3]['PDepth_mm'].max() ],
                  [ gDDict[4]['PDepth_mm'].max() ],
                  [ gDDict[5]['PDepth_mm'].max() ],
                  [ gDDict[6]['PDepth_mm'].max() ],
                  [ gDDict[7]['PDepth_mm'].max() ],
                  [ gDDict[8]['PDepth_mm'].max() ],
                  [ gDDict[9]['PDepth_mm'].max() ],
                  [ gDDict[10]['PDepth_mm'].max() ],
                  [ gDDict[11]['PDepth_mm'].max() ],
                  [ gDDict[12]['PDepth_mm'].max() ],
                ]
    # min
    MinLister = [ [ gDDict[1]['PDepth_mm'].min() ],
                  [ gDDict[2]['PDepth_mm'].min() ],
                  [ gDDict[3]['PDepth_mm'].min() ],
                  [ gDDict[4]['PDepth_mm'].min() ],
                  [ gDDict[5]['PDepth_mm'].min() ],
                  [ gDDict[6]['PDepth_mm'].min() ],
                  [ gDDict[7]['PDepth_mm'].min() ],
                  [ gDDict[8]['PDepth_mm'].min() ],
                  [ gDDict[9]['PDepth_mm'].min() ],
                  [ gDDict[10]['PDepth_mm'].min() ],
                  [ gDDict[11]['PDepth_mm'].min() ],
                  [ gDDict[12]['PDepth_mm'].min() ],
                ]
    # variance
    VarLister = [ [ gDDict[1]['PDepth_mm'].var() ],
                  [ gDDict[2]['PDepth_mm'].var() ],
                  [ gDDict[3]['PDepth_mm'].var() ],
                  [ gDDict[4]['PDepth_mm'].var() ],
                  [ gDDict[5]['PDepth_mm'].var() ],
                  [ gDDict[6]['PDepth_mm'].var() ],
                  [ gDDict[7]['PDepth_mm'].var() ],
                  [ gDDict[8]['PDepth_mm'].var() ],
                  [ gDDict[9]['PDepth_mm'].var() ],
                  [ gDDict[10]['PDepth_mm'].var() ],
                  [ gDDict[11]['PDepth_mm'].var() ],
                  [ gDDict[12]['PDepth_mm'].var() ],
                ]
    # skew
    SkewLister = [ [ gDDict[1]['PDepth_mm'].skew() ],
                  [ gDDict[2]['PDepth_mm'].skew() ],
                  [ gDDict[3]['PDepth_mm'].skew() ],
                  [ gDDict[4]['PDepth_mm'].skew() ],
                  [ gDDict[5]['PDepth_mm'].skew() ],
                  [ gDDict[6]['PDepth_mm'].skew() ],
                  [ gDDict[7]['PDepth_mm'].skew() ],
                  [ gDDict[8]['PDepth_mm'].skew() ],
                  [ gDDict[9]['PDepth_mm'].skew() ],
                  [ gDDict[10]['PDepth_mm'].skew() ],
                  [ gDDict[11]['PDepth_mm'].skew() ],
                  [ gDDict[12]['PDepth_mm'].skew() ],
                ]
    # kurtosis
    KurtLister = [ [ gDDict[1]['PDepth_mm'].kurtosis() ],
                  [ gDDict[2]['PDepth_mm'].kurtosis() ],
                  [ gDDict[3]['PDepth_mm'].kurtosis() ],
                  [ gDDict[4]['PDepth_mm'].kurtosis() ],
                  [ gDDict[5]['PDepth_mm'].kurtosis() ],
                  [ gDDict[6]['PDepth_mm'].kurtosis() ],
                  [ gDDict[7]['PDepth_mm'].kurtosis() ],
                  [ gDDict[8]['PDepth_mm'].kurtosis() ],
                  [ gDDict[9]['PDepth_mm'].kurtosis() ],
                  [ gDDict[10]['PDepth_mm'].kurtosis() ],
                  [ gDDict[11]['PDepth_mm'].kurtosis() ],
                  [ gDDict[12]['PDepth_mm'].kurtosis() ],
                ]
    # counts
    CntLister = [ [ gDDict[1]['PDepth_mm'].count() ],
                  [ gDDict[2]['PDepth_mm'].count() ],
                  [ gDDict[3]['PDepth_mm'].count() ],
                  [ gDDict[4]['PDepth_mm'].count() ],
                  [ gDDict[5]['PDepth_mm'].count() ],
                  [ gDDict[6]['PDepth_mm'].count() ],
                  [ gDDict[7]['PDepth_mm'].count() ],
                  [ gDDict[8]['PDepth_mm'].count() ],
                  [ gDDict[9]['PDepth_mm'].count() ],
                  [ gDDict[10]['PDepth_mm'].count() ],
                  [ gDDict[11]['PDepth_mm'].count() ],
                  [ gDDict[12]['PDepth_mm'].count() ],
                ]
    # process stats to dataframe
    DataDict = dict()
    DataDict["H1D_Ave"] = np.array( [x[0] for x in AveLister], np.float32 )
    DataDict["H1D_Med"] = np.array( [x[0] for x in MedianLister], np.float32 )
    DataDict["H1D_Max"] = np.array( [x[0] for x in MaxLister], np.float32 )
    DataDict["H1D_Min"] = np.array( [x[0] for x in MinLister], np.float32 )  
    DataDict["H1D_Var"] = np.array( [x[0] for x in VarLister], np.float32 )
    DataDict["H1D_Skew"] = np.array( [x[0] for x in SkewLister], np.float32 )
    DataDict["H1D_Kurt"] = np.array( [x[0] for x in KurtLister], np.float32 )
    DataDict["H1D_Cnt"] = np.array( [x[0] for x in CntLister], np.float32 )
    SumCompDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write out to a spreadsheet
    OutFileFP = os.path.normpath( os.path.join( OUT_DIR, "%s_WG_PDepth_Proj_G%d.xlsx" % (OUT_ROOT, gG) ) )
    with pd.ExcelWriter( OutFileFP, engine="openpyxl" ) as writer:
        SumCompDF.to_excel( writer, sheet_name="H1D_Summary" )
        DaQsDF.to_excel( writer, sheet_name="H1_Data" )
    # end of with
# end of for

Save out pickle files so that can do the plotting in a different notebook

In [34]:
for gG in LOCA_KEYS:
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_1.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][1].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_2.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][2].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_3.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][3].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_4.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][4].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_5.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][5].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_6.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][6].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_7.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][7].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_8.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][8].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_9.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][9].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_10.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][10].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_11.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][11].to_pickle( OutFiler, compression='zip' )
    OutFiler = os.path.normpath( os.path.join( OUT_DIR, "%s_H1DPDepth_G%d_12.pickle" % (OUT_ROOT, gG) ) )
    H1DDict[gG][12].to_pickle( OutFiler, compression='zip' )
# end for